In [1]:
import gzip
import random
import csv
from collections import defaultdict
from sklearn import linear_model
import math

In [2]:
path="trainInteractions.csv.gz"

In [3]:
def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [4]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [5]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [6]:
def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [7]:
dataset = list(readCSV(path))

In [8]:
dataset[0]

('88348277',
 '03969194',
 {'user_id': '88348277',
  'recipe_id': '03969194',
  'date': '2004-12-23',
  'rating': '5'})

In [9]:
itemsPerUser = defaultdict(set)
usersPerItem=defaultdict(set)
itemSet=set([d[1] for d in dataset])

In [10]:
for d in dataset:
    user,item = d[0], d[1]
    itemsPerUser[user].add(item)
    usersPerItem[item].add(user)

In [32]:
dataset2 = list(parse("trainRecipes.json.gz"))

In [12]:
ingsPerItem = defaultdict(set)
itemsPerIng = defaultdict(set)

In [13]:
for d in dataset2:
    r = d['recipe_id']
    for i in d['ingredients']:
        ingsPerItem[r].add(i)
        itemsPerIng[i].add(r)

1. Popularity and Jaccard similarity

In [14]:
def build_validate_set(dataset):
    validate_set=[]
    random.seed(50)
    for d in dataset:
        positive_entry=[d[0],d[1],1]
        negative_entry_item_set=itemSet.difference(itemsPerUser[d[0]])
        random_item=random.choice(list(negative_entry_item_set))
        negative_entry=[d[0],random_item,0]
        validate_set.append(positive_entry)
        validate_set.append(negative_entry)
    return validate_set

In [15]:
def build_train_set(dataset):
    train_set=[]
    for d in dataset:
        positive_entry=[d[0],d[1],d[2]]
        train_set.append(positive_entry)
    return train_set

In [25]:
def mostSimilar(i):
    ings = ingsPerItem[i]
    similarities = []
    for i2 in ingsPerItem: # For all items
        if i == i2: continue # other than the query
        sim = Jaccard(ings, ingsPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(key = lambda x: (-x[0], x[1]),reverse=False)
    for d in similarities:
        if d[1] in itemSet:
            return d[1]
    return dataset[0][1]

In [20]:
# def Pearson(i1, i2):
#     # Between two items
#     iBar1 = itemAverages[i1]
#     iBar2 = itemAverages[i2]
#     inter = usersPerItem[i1].intersection(usersPerItem[i2])
#     numer = 0
#     denom1 = 0
#     denom2 = 0
#     for u in inter:
#         numer += (ratingDict[(u,i1)] - iBar1)*(ratingDict[(u,i2)] - iBar2)
#     for u in usersPerItem[i1]:
#         denom1 += (ratingDict[(u,i1)] - iBar1)**2
#     for u in usersPerItem[i2]:
#         denom2 += (ratingDict[(u,i2)] - iBar2)**2
#     denom = math.sqrt(denom1) * math.sqrt(denom2)
#     if denom == 0: return 0
#     return numer / denom

In [14]:
train_set=build_train_set(dataset[:499000])
validate_set=build_validate_set(dataset[499000:500000])

In [21]:
def jaccardPopularityModel(train_set, test_set, pt=0.6):
    # calculate most popular set in train_set
    recipeCount = defaultdict(int)
    totalCooked = 0
    for d in train_set:
        recipeCount[d[1]] += 1
        totalCooked += 1

    mostPopular = [(recipeCount[x], x) for x in recipeCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalCooked*pt: break

    # evalute on test_set
    total_size=len(test_set)
    correct_size=0
    for i in range(total_size):
        sample=test_set[i]
        item=sample[1]
        user=sample[0]
        predict=0

        if item in return1:
            predict=1
        else:
            
#             maxSim1=0
#             for u in usersPerItem[item]:
#                 sim=Jaccard(itemsPerUser[u],itemsPerUser[user])
#                 maxSim1=max(maxSim1,sim)
#             print("1:")
#             print(maxSim1)
#             maxSim2=0
#             for d in itemsPerUser[user]:
#                 sim=Jaccard(usersPerItem[d],usersPerItem[item])
#                 maxSim2=max(maxSim2,sim)
#             print("2:")
#             print(maxSim2)
#             if maxSim2>0.1 and maxSim1>0.01:
#                 predict=1
#             maxSim1=0
#             for d in itemsPerUser[user]:
#                 sim=Pearson1(d,item)
#                 maxSim1=max(maxSim1,sim)
#             print("1:")
#             print(maxSim1)
            
            maxSim=0
            for d in itemsPerUser[user]:
                sim=Pearson(d,item)
                maxSim2=max(maxSim,sim)
            print("pearson:")
            print(maxSim)
            
            if maxSim2>0.1:
                predict=1
                
                
        if predict==sample[2]:
            correct_size+=1
    accuracy=correct_size/total_size
    print([accuracy])

In [22]:
jaccardPopularityModel(train_set,validate_set,0.694)

pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.5477225575051661
pearson:
0
pearson:
0
pearson:
0
pearson:
0.059085806446138964
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.10368527442368827
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.3333333333333332
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.6123724356957945
pearson:
0.8333333333333337
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.8164965809277259
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.5193819785289564
pearson:
0
pearson:
0
pearson:
0.20412414523193148
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.6846531968814575
pearson:
0
pearson:
0
pearson:
0
pearson:
0.6123724356957945
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearso

pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.016835875742536904
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.6149186938124421
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.13693063937629132
pearson:
0
pearson:
0
pearson:
0
pearson:
0.4166666666666667
pearson:
0
pearson:
0
pearson:
0
pearson:
0.28867513459481287
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.3091734712004211
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.6546536707079768
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.35355339059327373
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.23570226039551598
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.6324555320336757
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0
pearson:
0.577350269189626
pearson:
0.1195228609334392

Upload to kaggle:

In [80]:
def jaccardPopularityModel(train_set, pt=0.6):
    print("predicting....")

    # calculate most popular set in train_set
    recipeCount = defaultdict(int)
    totalCooked = 0
    for d in train_set:
        recipeCount[d[1]] += 1
        totalCooked += 1

    mostPopular = [(recipeCount[x], x) for x in recipeCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalCooked*pt: break

    predictions = open("predictions_Made.txt", 'w')
    for l in open("stub_Made.txt"):
        if l.startswith("user_id"):
            predictions.write(l)
            continue
        user,item = l.strip().split('-')
        item1=item
        predict=0
            

        if item in return1:
            predict=1
        else:
            
            if item not in itemSet:
                predict=0
            else:
#                 maxSim=0
#                 for d in itemsPerUser[user]:
#                     sim=Pearson(d,item)
#                     maxSim=max(maxSim,sim)
#                 print("Pearson:")
#                 print(maxSim)

                maxSim1=0
                for u in usersPerItem[item]:
                    sim=Jaccard(itemsPerUser[u],itemsPerUser[user])
                    maxSim1=max(maxSim1,sim)
                print("1:")
                print(maxSim1)
                maxSim2=0
                for d in itemsPerUser[user]:
                    sim=Jaccard(usersPerItem[d],usersPerItem[item])
                    maxSim2=max(maxSim2,sim)
                print("2:")
                print(maxSim2)
                if maxSim1>0.01 and maxSim2 > 0.05:
                    predict=1 

        if predict==1:
            predictions.write(user + '-' + item1 + ",1\n")
        else:
            predictions.write(user + '-' + item1 + ",0\n")
    predictions.close()
    print("predicting finished!")

In [81]:
train_set=build_train_set(dataset)

In [82]:
jaccardPopularityModel(train_set,0.595)

predicting....
1:
0.0036407766990291263
2:
0.017543859649122806
1:
0.027188081936685288
2:
0.2857142857142857
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.0053388090349075976
2:
0.5
1:
0.03212468193384224
2:
0.25
1:
0.002390438247011952
2:
0.09090909090909091
1:
0.0023584905660377358
2:
0.2
1:
0.005631687629059508
2:
0.3333333333333333
1:
0.01232114467408585
2:
0.5
1:
0.005376344086021506
2:
0.008849557522123894
1:
0
2:
0
1:
0.0065420560747663555
2:
0.25
1:
0.00034083162917518747
2:
0.029411764705882353
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.002207505518763797
2:
0.5
1:
0
2:
0
1:
0.0007423904974016332
2:
0.06666666666666667
1:
0.002824858757062147
2:
0.024390243902439025
1:
0.004629629629629629
2:
0.2
1:
0.00020824656393169514
2:
0.3333333333333333
1:
0.009009009009009009
2:
0.005154639175257732
1:
0
2:
0
1:
0
2:
0
1:
0.01690039554117224
2:
0.25
1:
0.016735028712059064
2:
0.5
1:
0.004098360655737705
2:
0.017857142857142856
1:
0
2:
0
1:
0.002551020408163265
2:
0.011363636363636364
1:
0.00356188780053

0.25
1:
0.020985401459854013
2:
0.5
1:
0.004075846181109339
2:
0.5
1:
0.0013844023996308261
2:
0.3333333333333333
1:
0
2:
0
1:
0.0027548209366391185
2:
0.25
1:
0
2:
0
1:
0
2:
0
1:
0.004392386530014641
2:
0.02702702702702703
1:
0.004461440407903123
2:
0.125
1:
0.0007936507936507937
2:
0.002183406113537118
1:
0
2:
0
1:
0.007470119521912351
2:
0.16666666666666666
1:
0
2:
0
1:
0.004124656278643446
2:
0.1111111111111111
1:
0
2:
0
1:
0.0035211267605633804
2:
0.014492753623188406
1:
0
2:
0
1:
0.0007974481658692185
2:
0.2
1:
0.0031645569620253164
2:
0.002369668246445498
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.0023923444976076554
2:
0.0029498525073746312
1:
0.012631233595800526
2:
0.5
1:
0
2:
0
1:
0.0015015015015015015
2:
0.16666666666666666
1:
0.014150943396226415
2:
0.01098901098901099
1:
0
2:
0
1:
0.0029850746268656717
2:
0.14285714285714285
1:
0.0006906077348066298
2:
0.021739130434782608
1:
0
2:
0
1:
0.0019723865877712033
2:
0.5
1:
0.0011876484560570072
2:
0.058823529411764705
1:
0
2:
0
1:
0.00

2:
0
1:
0.0015408320493066256
2:
0.007936507936507936
1:
0.005009392611145898
2:
0.1
1:
0.009831029185867896
2:
0.25
1:
0
2:
0
1:
0.0015552099533437014
2:
0.2
1:
0
2:
0
1:
0
2:
0
1:
0.001941747572815534
2:
0.08333333333333333
1:
0.00732421875
2:
0.14285714285714285
1:
0.005529953917050691
2:
0.14285714285714285
1:
0.0392156862745098
2:
0.011494252873563218
1:
0
2:
0
1:
0.013107416879795396
2:
0.25
1:
0.005490196078431373
2:
0.3333333333333333
1:
0.006211180124223602
2:
0.2
1:
0
2:
0
1:
0.0015290519877675841
2:
0.05555555555555555
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.0027100271002710027
2:
0.25
1:
0.0018484288354898336
2:
0.05555555555555555
1:
0
2:
0
1:
0.004038772213247173
2:
0.3333333333333333
1:
0.000474158368895211
2:
0.09090909090909091
1:
0.00425531914893617
2:
0.058823529411764705
1:
0.0007215007215007215
2:
0.020833333333333332
1:
0
2:
0
1:
0
2:
0
1:
0.010569852941176471
2:
0.3333333333333333
1:
0
2:
0
1:
0
2:
0
1:
0.01485148514851485
2:
0.058823529411764705
1:
0.0029282576866764

2:
0.1111111111111111
1:
0.006944444444444444
2:
0.02702702702702703
1:
0.023393984404010398
2:
0.5
1:
0
2:
0
1:
0
2:
0
1:
0.011581067472306143
2:
0.3333333333333333
1:
0.01885057471264368
2:
0.25
1:
0.006024096385542169
2:
0.16666666666666666
1:
0
2:
0
1:
0.00487012987012987
2:
0.1
1:
0
2:
0
1:
0
2:
0
1:
0.03352189398701027
2:
0.3333333333333333
1:
0.0016560861164780568
2:
0.06666666666666667
1:
0.0076045627376425855
2:
0.006622516556291391
1:
0
2:
0
1:
0.000687757909215956
2:
0.3333333333333333
1:
0.0015220700152207
2:
0.024390243902439025
1:
0.001182033096926714
2:
0.013513513513513514
1:
0.008869179600886918
2:
0.029411764705882353
1:
0.015518028297581013
2:
0.25
1:
0
2:
0
1:
0.006694720734506504
2:
0.3333333333333333
1:
0
2:
0
1:
0.0120415982484948
2:
0.2
1:
0
2:
0
1:
0
2:
0
1:
0.005680351148980119
2:
0.5
1:
0.0005955926146515784
2:
0.029411764705882353
1:
0.0005903187721369539
2:
0.0196078431372549
1:
0.001148105625717566
2:
0.08333333333333333
1:
0.00842548709847288
2:
0.2
1:
0.

2:
0.045454545454545456
1:
0.002304147465437788
2:
0.014705882352941176
1:
0.001669449081803005
2:
0.2
1:
0.0028694404591104736
2:
0.058823529411764705
1:
0.0032442748091603053
2:
0.5
1:
0
2:
0
1:
0
2:
0
1:
0.0026455026455026454
2:
0.1
1:
0.003303964757709251
2:
0.16666666666666666
1:
0.0006598482349059716
2:
0.125
1:
0
2:
0
1:
0.006097560975609756
2:
0.022727272727272728
1:
0
2:
0
1:
0
2:
0
1:
0.00849673202614379
2:
0.5
1:
0
2:
0
1:
0.012048192771084338
2:
0.023809523809523808
1:
0.0015313935681470138
2:
0.16666666666666666
1:
0.003157894736842105
2:
0.014492753623188406
1:
0
2:
0
1:
0.016537467700258397
2:
0.2
1:
0.0011001100110011
2:
0.010526315789473684
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.011661807580174927
2:
0.4
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.006493506493506494
2:
0.3333333333333333
1:
0.005092761004001455
2:
0.5
1:
0.004646840148698885
2:
0.03333333333333333
1:
0
2:
0
1:
0
2:
0
1:
0.030303030303030304
2:
0.006329113924050633
1:
0
2:
0
1:
0.0072405470635559

...

2:
0.3333333333333333
1:
0
2:
0
1:
0.027777777777777776
2:
0.2
1:
0.006361323155216285
2:
0.14285714285714285
1:
0.0036809815950920245
2:
0.25
1:
0.004572299485616308
2:
0.3333333333333333
1:
0
2:
0
1:
0.004016064257028112
2:
0.1111111111111111
1:
0.00535475234270415
2:
0.3333333333333333
1:
0.002183406113537118
2:
0.16666666666666666
1:
0.0014527845036319612
2:
0.1111111111111111
1:
0.0008080808080808081
2:
0.038461538461538464
1:
0.0034522439585730723
2:
0.125
1:
0
2:
0
1:
0.004662004662004662
2:
0.2
1:
0.010752688172043012
2:
0.014705882352941176
1:
0.025657071339173967
2:
0.25
1:
0.008547008547008548
2:
0.1
1:
0
2:
0
1:
0.0013422818791946308
2:
0.0029585798816568047
1:
0
2:
0
1:
0
2:
0
1:
0.004032258064516129
2:
0.008695652173913044
1:
0
2:
0
1:
0.002311111111111111
2:
0.5
1:
0.0044742729306487695
2:
0.006993006993006993
1:
0
2:
0
1:
0
2:
0
1:
0.003281378178835111
2:
0.25
1:
0.01687289088863892
2:
0.16666666666666666
1:
0.002927400468384075
2:
0.5
1:
0.002807862013638187
2:
0.14285

2:
0.2
1:
0
2:
0
1:
0.0012195121951219512
2:
0.006289308176100629
1:
0.015562248995983935
2:
0.25
1:
0
2:
0
1:
0
2:
0
1:
0.006124234470691163
2:
0.3333333333333333
1:
0.007462686567164179
2:
0.023255813953488372
1:
0.001996007984031936
2:
0.07142857142857142
1:
0
2:
0
1:
0.004024144869215292
2:
0.05555555555555555
1:
0
2:
0
1:
0.004283965728274173
2:
0.16666666666666666
1:
0.012290502793296089
2:
0.25
1:
0
2:
0
1:
0.020915896081021578
2:
0.4
1:
0
2:
0
1:
0
2:
0
1:
0.002777777777777778
2:
0.5
1:
0
2:
0
1:
0
2:
0
1:
0.0009337068160597573
2:
0.07692307692307693
1:
0.005763688760806916
2:
0.034482758620689655
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.015329125338142471
2:
0.25
1:
0
2:
0
1:
0
2:
0
1:
0.003616636528028933
2:
0.022727272727272728
1:
0
2:
0
1:
0.005128205128205128
2:
0.006134969325153374
1:
0.002857142857142857
2:
0.08333333333333333
1:
0.0012315270935960591
2:
0.1111111111111111
1:
0
2:
0
1:
0.004243281471004243
2:
0.038461538461538464
1:
0
2:
0
1:
0
2:
0
1:
0.02

2:
0
1:
0.0042451578668081715
2:
0.3333333333333333
1:
0.00130718954248366
2:
0.16666666666666666
1:
0.003432796408766834
2:
0.5
1:
0.006887052341597796
2:
0.25
1:
0.012987012987012988
2:
0.00546448087431694
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.0017211703958691911
2:
0.005714285714285714
1:
0
2:
0
1:
0
2:
0
1:
0.0031466331025802393
2:
0.25
1:
0
2:
0
1:
0.00646551724137931
2:
0.02040816326530612
1:
0.00046948356807511736
2:
0.0136986301369863
1:
0.004591836734693878
2:
0.2
1:
0.000779423226812159
2:
0.05
1:
0.0007468259895444362
2:
0.007633587786259542
1:
0
2:
0
1:
0
2:
0
1:
0.017801590044936054
2:
0.5
1:
0.004807692307692308
2:
0.16666666666666666
1:
0.0036529680365296802
2:
0.25
1:
0
2:
0
1:
0
2:
0
1:
0.0019815059445178335
2:
0.14285714285714285
1:
0.0014388489208633094
2:
0.008695652173913044
1:
0
2:
0
1:
0.00881057268722467
2:
0.03571428571428571
1:
0.007023230686115613
2:
0.2
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.008778346744696415
2:
0.16666666666666666
1:
0.0185983

2:
0.047619047619047616
1:
0
2:
0
1:
0.0020768431983385254
2:
0.1
1:
0.0034812880765883376
2:
0.2
1:
0.0056179775280898875
2:
0.0064516129032258064
1:
0.0013679890560875513
2:
0.0016155088852988692
1:
0.0008203445447087777
2:
0.06666666666666667
1:
0.0003429355281207133
2:
0.125
1:
0
2:
0
1:
0.006557377049180328
2:
0.034482758620689655
1:
0.006952491309385863
2:
0.3333333333333333
1:
0
2:
0
1:
0
2:
0
1:
0.005192107995846314
2:
0.0625
1:
0
2:
0
1:
0
2:
0
1:
0.008600469116497263
2:
0.2
1:
0.0027247956403269754
2:
0.1111111111111111
1:
0.002379535990481856
2:
0.25
1:
0
2:
0
1:
0.000931098696461825
2:
0.25
1:
0.0014947683109118087
2:
0.05555555555555555
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.000594883997620464
2:
0.08333333333333333
1:
0
2:
0
1:
0.004545454545454545
2:
0.01694915254237288
1:
0.01753393665158371
2:
0.3333333333333333
1:
0
2:
0
1:
0.01282051282051282
2:
0.02702702702702703
1:
0.001201923076923077
2:
0.16666666666666666
1:
0
2:
0
1:
0.0025974025974025974
2:
0.

2:
0.017241379310344827
1:
0
2:
0
1:
0
2:
0
1:
0.005104408352668214
2:
0.25
1:
0
2:
0
1:
0.004
2:
0.25
1:
0.0016750418760469012
2:
0.05
1:
0.00410958904109589
2:
0.25
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.0029644268774703555
2:
0.16666666666666666
1:
0.0030911901081916537
2:
0.3333333333333333
1:
0.006772009029345372
2:
0.047619047619047616
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.007780659503519822
2:
0.25
1:
0
2:
0
1:
0.004971002485501243
2:
0.16666666666666666
1:
0
2:
0
1:
0
2:
0
1:
0
2:
0
1:
0.004833375731366065
2:
0.2857142857142857
1:
0.0027472527472527475
2:
0.14285714285714285
1:
0.001984126984126984
2:
0.02127659574468085
1:
0.0024096385542168677
2:
0.004739336492890996
1:
0.008849557522123894
2:
0.012195121951219513
1:
0
2:
0
1:
0.0026980150317980345
2:
0.3333333333333333
1:
0.0035714285714285713
2:
0.00546448087431694
1:
0
2:
0
1:
0.004050405040504051
2:
0.25
1:
0.004878048780487805
2:
0.010416666666666666
1:
0.0004051863857374392
2:
0.06666666666666667
1:
0.00345489443378119
2:
0.16

2. Classification

In [ ]:
# import sklearn
# from sklearn import linear_model

In [ ]:
# def build_train_set(dataset):
#     itemsPerUser = defaultdict(set)
#     usersPerItem=defaultdict(set)
#     for d in dataset:
#         user,item = d[0], d[1]
#         itemsPerUser[user].add(item)
#         usersPerItem[item].add(user)
#     train_set=[]
#     random.seed(50)
#     for d in dataset:
#         positive_entry=[d[0],d[1],1]
#         negative_entry_item_set=itemSet.difference(itemsPerUser[d[0]])
#         random_item=random.choice(list(negative_entry_item_set))
#         negative_entry=[d[0],random_item,0]
#         train_set.append(positive_entry)
#         train_set.append(negative_entry)
#     return train_set

In [ ]:
# train_set=build_train_set(dataset)

In [ ]:
# itemsPerUser = defaultdict(set)
# usersPerItem=defaultdict(set)
# for d in train_set:
#     user,item = d[0], d[1]
#     itemsPerUser[user].add(item)
#     usersPerItem[item].add(user)

# itemSet=set([d[1] for d in train_set])
# userSet=set([d[0] for d in train_set])

In [ ]:
# # calculate most popular set in train_set
# recipeCount = defaultdict(int)
# totalCooked = 0
# for d in train_set:
#     recipeCount[d[1]] += 1
#     totalCooked += 1

# mostPopular = [(recipeCount[x], x) for x in recipeCount]
# mostPopular.sort()
# mostPopular.reverse()

# return1 = set()
# count = 0
# for ic, i in mostPopular:
#     count += ic
#     return1.add(i)
#     if count > totalCooked*0.6: break

In [ ]:
# def feature(user, item):
#     feat = [1]
#     if item in return1:
#         feat.append(1)
#     else:
#         feat.append(0)    
#     maxSim=0
#     for d in itemsPerUser[user]:
#         sim=Jaccard(usersPerItem[d],usersPerItem[item])
#         maxSim=max(maxSim,sim)
#     feat.append(maxSim-0.01)
#     return feat

In [ ]:
# X=[]
# y=[]

In [ ]:
# for d in train_set:
#     user=d[0]
#     item=d[1]
#     X.append(feature(user,item))
#     y.append(d[2]==1)

In [ ]:
# model = sklearn.linear_model.LogisticRegression()
# model.fit(X, y)

In [ ]:
# X_test=[]

In [ ]:
# predictions = open("predictions_Made.txt", 'w')
# for l in open("stub_Made.txt"):
#     if l.startswith("user_id"):
#         predictions.write(l)
#         continue
#     user,item = l.strip().split('-')
#     X_test.append(feature(user,item))
# y_predict=model.predict(X_test)

# i=0
# for l in open("stub_Made.txt"):
#     if l.startswith("user_id"):
#         continue
#     user,item = l.strip().split('-')
#     predict=y_predict[i]
#     if predict==True:
#         predictions.write(user + '-' + item + ",1\n")
#     else:
#         predictions.write(user + '-' + item + ",0\n")
#     i+=1
# predictions.close()
# print("predicting finished!")